In [29]:
import pandas as pd
import numpy as np
import os
import pybaseball as pyb
from unidecode import unidecode

In [25]:
ids_df = pd.read_csv("data/ids.csv", index_col=False)

In [ ]:
# function removes unnamed col
def removeUnnamedCol(df):
    # returns columns that do not start with Unnamed
    return df.loc[:, ~df.columns.str.startswith("Unnamed")]

In [28]:
id_lookup_dict = dict(zip(ids_df.MLBID, ids_df.PLAYERNAME))
print(id_lookup_dict.get(592450))

Aaron Judge
